In [ ]:
import glob
import os.path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn import datasets, linear_model, ensemble, neural_network
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from pathlib import Path

In [ ]:
CONFIG_FILE = '../entsoe-data.config'

if not os.path.exists(CONFIG_FILE):
    download_dir = input('Path to ENTSO-E data folder: ')
    if not os.path.isdir(download_dir):
        raise RuntimeError(f'Invalid download_dir, please run cell again: {download_dir}')
    with open(CONFIG_FILE, 'w') as f:
        f.write(download_dir)
else:
    with open(CONFIG_FILE) as f:
        download_dir = f.read()
        
# Clear the output after this cell if you want to aovid having your path in the notebook (or execute it twice)!

In [ ]:

def read_single_csv_entso_e(file):
    return pd.read_csv(file, sep='\t', encoding='utf-16', parse_dates=["DateTime"])


def load_complete_entso_e_data(directory):
    pattern = Path(directory) / '*.csv'
    files = glob.glob(str(pattern))

    if not files:
        raise ValueError(f"No files found when searching in {pattern}, wrong directory?")
    
    print(f'Concatenating {len(files)} csv files...')

    each_csv_file = [read_single_csv_entso_e(file) for file in files]
    data = pd.concat(each_csv_file, ignore_index=True)

    data = data.sort_values(by=["AreaName", "DateTime"])
    data = data.set_index("DateTime")

    print("Loading done.")

    return data


power_demand = load_complete_entso_e_data(download_dir)

In [ ]:
def get_hourly_country_data(data, country):
    ret_data = data[data["AreaName"] == country].interpolate() # data may contain NAs, therefore inteprolate
    ret_data = ret_data.resample("1h").mean().interpolate() # not all hours may be  complete 
                                                            # (i.e. some last 15 minutes are lacking, therefore another inpolation here)

    return ret_data

power_demand_at_hourly = get_hourly_country_data(power_demand, "Austria")["2015-01-01":"2019-12-31"]

## Today's goal

We want to understand if electricity load is lower than expected due to the Corona Lockdown. We therefore have to know which electricity load we should have expected without the lockdown.

We do so by fitting a function to the electricity load, i.e. $y=f(x_1, x_2, ..., x_n)$. $y$ is the output feature, in our case the load. $f$ is some function depending on some $x_i$. We call the $x_i$ input features in the following.

Some useful links
- [A very brief introduction to machine learning concepts](https://towardsdatascience.com/introduction-to-machine-learning-for-beginners-eed6024fdb08)
- [A more lengthy introduction to machine learning with good visuals](https://www.toptal.com/machine-learning/machine-learning-theory-an-introductory-primer)
- [An introduction to machine learning regression algorithms](https://medium.com/datadriveninvestor/regression-in-machine-learning-296caae933ec)


## Exercise 1

What could we fit to the load - when you think of the last lecture? Which function could be fit to the model? Which data could be used to describe that function? 

In [ ]:
# Put some potential input features here... 

Let's do it!

We use a Random Forest model for mapping input features to output features. Wanna know what a Random Forest is? [Check this](https://medium.com/@williamkoehrsen/random-forest-simple-explanation-377895a60d2d)

In [ ]:
Y = power_demand_at_hourly["TotalLoadValue"].values
X = power_demand_at_hourly.index.month.values[:, np.newaxis]

forest_simple = ensemble.RandomForestRegressor()

# Train the model using the training sets
forest_simple.fit(X, Y)

prediction = forest_simple.predict(X)



In [ ]:
def plot_prediction(Y, prediction, alpha=1):
    plt.plot(Y, label="Observation")
    plt.plot(prediction, label="Prediction", alpha=alpha)
    plt.xlabel("Time")
    plt.ylabel("Load (MW)")
    plt.legend()
    
plot_prediction(Y, prediction)

Hm... how could we do even better perhaps?

In [ ]:
X = np.array([power_demand_at_hourly.index.month.values,
     power_demand_at_hourly.index.weekday.values,
     power_demand_at_hourly.index.hour.values]).T

forest_all_time_scales = ensemble.RandomForestRegressor()

# Train the model using the training sets
forest_all_time_scales.fit(X, Y)

predicted = forest_all_time_scales.predict(X)

plot_prediction(Y, predicted, alpha=0.5)


Let's zoom in a bit...

In [ ]:
plot_prediction(Y[400:500], predicted[400:500])


Let's do it a little bit more correct:
you should train your data on a training data set and test it on a test data set.
If you do not do that, you may find an extremely good fit, but when you use new data on the algorithm, it may fail!


In [ ]:
X_training, X_test, Y_training, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
forest_split_set = ensemble.RandomForestRegressor()

# Train the model using the training sets
forest_split_set.fit(X_training, Y_training)

prediction = forest_split_set.predict(X_test)

plt.plot(Y_test)
plt.plot(prediction, alpha=0.5)
plt.ylabel("Load (MW)")

## Exercise 2

Why does this figure look different from the other load figures? How else could you plot the data to see if the fit is good?

In [ ]:
# Put some reasons here

In [ ]:
# Do another plot

Let's calculate a quality parameter for the fit...

In [ ]:
prediction_training = forest_split_set.predict(X_training)
prediction_test = forest_split_set.predict(X_test)

print(r2_score(Y_training, prediction_training))
print(r2_score(Y_test, prediction_test))

Wowh, that's pretty good. And we are just using time information! That means we have pretty regular data.

## Exercise 3

Now redo the model, but instead of months use dayofyear for seasonal adjustment. Calculate R2 if you predict with training data and calculate R2 if you predict with test data. What do you think, is quality improved?

Pretty amazing fit! I think we can work with the model.


Now let's look how this worked for 2020...

In [ ]:
power_demand_at_hourly_2020 =  get_hourly_country_data(power_demand, "Austria")["2020-01-01":"2020-05-31"]

In [ ]:
X = np.array([power_demand_at_hourly.index.dayofyear.values,
     power_demand_at_hourly.index.weekday.values,
     power_demand_at_hourly.index.hour.values]).T

X_training, X_test, Y_training, Y_test = train_test_split(X, Y, test_size=0.2)

forest_dayofyear = ensemble.RandomForestRegressor()

# Train the model using the training sets
forest_dayofyear.fit(X_training, Y_training)

prediction_training = forest_dayofyear.predict(X_training)

prediction_test = forest_dayofyear.predict(X_test)

print(r2_score(Y_training, prediction_training))
print(r2_score(Y_test, prediction_test))

In [ ]:
X_2020 = np.array([power_demand_at_hourly_2020.index.dayofyear.values,
     power_demand_at_hourly_2020.index.weekday.values,
     power_demand_at_hourly_2020.index.hour.values]).T

prediction_2020 = forest_dayofyear.predict(X_2020)

plot_prediction(power_demand_at_hourly_2020["TotalLoadValue"].values, prediction_2020, alpha=0.5)

## Exercise 4

Calculate the proportion of predicted vs. observed generation on a monthly basis and plot it! Hint: it is easier if you assign a new column to ```power_demand_at_hourly_2020``` with the predicted values from ```pred_2020```